In [27]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [28]:
archivos = [
    'becquer.txt',
    'machado.txt'
]

In [29]:
textos = []
etiquetas = []

In [30]:
for etiqueta, f in enumerate(archivos):
    print(f"{f} corresponde a {etiqueta}")

    with open(f, 'r', encoding='utf-8') as archivo:
        for line in archivo:
            line = line.rstrip().lower()
            if line:
                line = line.translate(str.maketrans('','',string.punctuation))
                textos.append(line)
                etiquetas.append(etiqueta)


becquer.txt corresponde a 0
machado.txt corresponde a 1


In [31]:
print(textos[:15])
print(etiquetas[:15])

['rimas', '    yo sé un himno gigante y extraño', 'que anuncia en la noche del alma una aurora', 'y estas páginas son de ese himno', 'cadencias que el aire dilata en las sombras', '    yo quisiera escribirlo del hombre', 'domando el rebelde mezquino idioma', 'con palabras que fuesen a un tiempo', 'suspiros y risas colores y notas', '   pero en vano es luchar que no hay cifra', 'capaz de encerrarlo y apenas  oh hermosa', 'si teniendo en mis manos las tuyas', 'pudiera al oído contártelo a solas', '    saeta que voladora', 'cruza arrojada al azar']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [32]:
train_text, test_text, Ytrain, Ytest = train_test_split(
    textos, etiquetas, test_size=0.25, random_state=42)

In [33]:
len(Ytrain), len(Ytest)

(1824, 609)

In [34]:
train_text[:15]

['        y andar andar',
 'y si les llega en sueños como un rumor distante',
 'despidiose el duelo',
 ' cuántas veces temblar sobre los vidrios',
 '    saeta que voladora',
 '       del cabezal de piedra',
 'cerré ya por que no entre',
 'que a un mismo tronco enlazadas',
 'agilitó su brazo acreditó su brío',
 'torno a ver sus pupilas llamear',
 'la espalda del moncayo',
 'sobre el olivar',
 'mas tengo en mi tristeza una alegría',
 'de portales con escudos',
 '      en las zarzas agudas']

In [35]:
Ytrain[:15]

[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0]

In [36]:
indice = 1
indicepalabras = {'<unk>' : 0}

In [37]:
for texto in train_text:
    tokens = texto.split()
    for token in tokens:
        if token not in indicepalabras:
            indicepalabras[token] = indice
            indice += 1

In [38]:
train_text_int = []

for texto in train_text:
    tokens = texto.split()
    linea_entero = [indicepalabras[token] for token in tokens]
    train_text_int. append(linea_entero)
len(train_text_int)

1824

In [39]:
test_text_int = []
for texto in test_text:
    tokens = texto.split()
    linea_entero = [indicepalabras.get(token, 0) for token in tokens]
    test_text_int. append(linea_entero)

len(test_text_int)

609

In [40]:
V = len(indicepalabras)
A0 = np.ones((V,V))
pi0 = np.ones(V)

A1 =  np.ones((V,V))
pi1 = np.ones(V)

In [41]:
def compute_counts(text_as_int, A, pi):
    for tokens in text_as_int:
        last_idx = None
        for idx in tokens:
            if last_idx is None:
                pi[idx] += 1
            else:
                A[last_idx, idx] += 1
            last_idx = idx

In [42]:
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y==0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y==1], A1, pi1)

In [43]:
# Normalizamos A y pi
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [44]:
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [45]:
logpi0

array([-8.27537637, -3.85653577, -8.27537637, ..., -8.27537637,
       -8.27537637, -8.27537637])

In [46]:
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, logp0, p1, logp1

(0.5630482456140351,
 -0.5743899607057185,
 0.4369517543859649,
 -0.8279324918440617)

In [47]:
class Classifier:
    def __init__(self, logAs, logpis, logpriors):
        self.logAs = logAs
        self.logpis = logpis
        self.logpriors = logpriors
        self.K = len(logpriors)
    def _compute_log_likelihood(self, input_, class_):
        logA = self.logAs[class_]
        logpi = self.logpis[class_]

        last_idx = None
        logprob = 0
        for idx in input_:
            if last_idx is None:
                logprob += logpi[idx]
            else:
                logprob += logA[last_idx, idx]
            
            last_idx = idx
        return logprob
    def predict(self, inputs):
        predictions = np.zeros(len(inputs))
        for i, input_ in enumerate(inputs):
            posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
                          for c in range(self.K)]
            pred = np.argmax(posteriors)
            predictions[i] = pred
        return predictions
        

In [48]:
clf =  Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

Ptrain = clf.predict(train_text_int)
print(f"Precisión del train: {np.mean(Ptrain == Ytrain)}")

Precisión del train: 0.9917763157894737


In [49]:
Ptest = clf.predict(test_text_int)
print(f"Precisión del test: {np.mean(Ptest == Ytest)}")

Precisión del test: 0.6962233169129721
